# ALEZ17 - Alaiz measuring campaign within the NEWA project
This notebook serves as an example for using the CSV_timeseries reader.
CSV_timeseries uses the pandas pd.read_csv function for loading data thus is compatible with any input that the function can read.

CSV_timeseries supports the variables dictionarry and custom lambda funtions for data processing.


## 1. Import dependencies

In [1]:
import pandas as pd 
import sys
import math

sys.path.append("../..") #path to the folder with lidaco
from lidaco.core.Builder import Builder



## 2. Convert the dataset

### 2.1 Define lambda funtions for data processing
The lambda funtions are aplied per row, so they are aware of all the variables in the row. They also have access to the section of the config defined for the variable they are supposed to generate. 

In [2]:
# return None value
def get_None(input_data_row, var_config):
    return None
# apply rotation to the sonic data based on the boom_angle
def rot(input_data_row, var_config):
    name_from = var_config['name_from']
    x_dim = y_dim = ""
    if name_from[-1] == 'U':
        x_dim = name_from
        y_dim = name_from[:-1]+'V'
        ret_u = True
    elif name_from[-1] == 'V':
        x_dim = name_from[:-1]+'U'
        y_dim = name_from
        ret_u = False
    elif name_from[-1] == 'X':
        x_dim = name_from
        y_dim = name_from[:-1]+'Y'
        ret_u = True
    elif name_from[-1] == 'Y':
        x_dim = name_from[:-1]+'X'
        y_dim = name_from
        ret_u = False

    x = input_data_row[x_dim]
    y = input_data_row[y_dim]
    
    angle_deg = var_config['boom_angle']
    angle = angle_deg/180 * 3.1415926
    
    U = x*math.cos(angle) - y*math.sin(angle)
    V = x*math.sin(angle) + y*math.cos(angle)
    
    if ret_u:
        return U
    else:
        return V

# reconstruct the timestamp in the correct format
import datetime
def fix_time(input_data_row):
    #print(input_data_row[0,0])
    time_str = str(input_data_row['Name'])
    
    ###  str: 201809300100,
    ###       012345678901234567890
    year    = int(time_str[0:4])
    month   = int(time_str[4:6])
    day     = int(time_str[6:8])
    hours   = int(time_str[8:10])
    minutes = int(input_data_row['MM'])
    seconds = int(input_data_row['SS'])
    m_sec   = (int(input_data_row['mS'])-32250)*1000
    #print(year, month, day, hours, minutes,seconds, m_sec)
    dt = datetime.datetime(year, month, day, hours, minutes,seconds, m_sec)

    seconds_since = (dt - datetime.datetime(1970,1,1)).total_seconds()
    return seconds_since

### 2.2 Sensors and towers setup 

Data based on the experiment reeport https://doi.org/10.5281/zenodo.3187482

In [3]:
# towers positions and id's
# table 8 in the reeport
d = {
    'id':  [2, 3,6,7], 
    'lon': [-1.572331536305404,-1.5633646394797014,-1.5713723837817706,-1.577016143668346], 
    'lat': [42.71567597338727,42.74385844765364,42.75001917791451,42.72497372458333] }
towers = pd.DataFrame(data=d)

# sensors heights 
heights = [10,20,40,60,80]

# sonics setup
# table 7 in the reeport
d =[[2,'Gill sonic',9.883,264.002,-0.408],
    [2,'Gill sonic',20.151,260.674,-0.246],
    [2,'Gill sonic',41.823,253.302,-1.089],
    [2,'Gill sonic',61.161,254.514,-1.27],
    [2,'Gill sonic',82.2,251.629,-1.255],
    [3,'Gill sonic',9.609,302.065,1.167],
    [3,'Gill sonic',20.209,298.249,0.615],
    [3,'Gill sonic',40.952,292.062,-2.995],
    [3,'Gill sonic',61.606,292.707,-1.945],
    [3,'Gill sonic',82.215,293.16,-0.573],
    [7,'Gill sonic',7.706,267.39,-0.143],
    [7,'Gill sonic',17.882,266.636,-1.58],
    [7,'Gill sonic',38.946,265.997,-0.564],
    [7,'Gill sonic',58.778,263.56,-1.107],
    [7,'Gill sonic',79.251,259.572,-1.782],
    [6,'Metek sonic',9.603,264.924,0.469],
    [6,'Metek sonic',20.12,261.603,-2.79],
    [6,'Metek sonic',41.227,255.266,-1.51],
    [6,'Metek sonic',60.595,251.351,-2.529]]
sonics = pd.DataFrame(data=d, columns = ['Mast', 'Sensor', 'height', 'Orientation','Tilt'])
sonics['height'] = sonics['height'].round(-1)

### 2.3 Variables definitons
Table with variables translation $(from, to, funtion_to_apply)$

In [4]:
variables_translation = [
    ('U', "eastward_wind", rot), #rotation here
    ('V', "northward_wind", rot),# rotation here
    ('W', "upward_air_velocity",None),
    ('T', "air_temperature",None),# TODO celsius_to_kalvin),
    ('st_code', "st_code",None),
    ('status', "status",None),
]
# tower 6 is special
t6_heights = [10,20,40,60]
t6_variables_translation = [
    ('SX', "eastward_wind", rot), #rotation here
    ('SY', "northward_wind", rot),# rotation here
    ('SZ', "upward_air_velocity",None),
    ('St', "air_temperature",get_None),
    ('St', "st_code",None),
    ('status', "status",None),
]  

### 2.4 Generate the settings for the converter
While it is both possible and encouraged to define the case setup using settings file, in this case the majority of the setup is code-generated. There are two reasons for that:
- passing custom funtions for processing the data,
- dealing with a large number of variables (close to a hundred in this case) which could be defined in a loop.

In [5]:
#base config stucture
config = {
    'parameters': {
        'time_from':'TimeStamp',
        'positions':[]
    },
}

#loop through all the towers
for _, tower in towers.iterrows(): 
    if tower["id"] == 6: # tower 6 is special
        variables = t6_variables_translation
        hs = t6_heights
    else:    
        variables = variables_translation
        hs = heights
    #loop through heights
    for height in hs:
        #generate variables names for a turbine
        variables_config = []
        # loop through variables
        for (variable_from, variable_to, f_apply) in variables:
            # config of a single variable
            variable_from = "Twr"+str(int(tower["id"]))+"_"+str(height)+"m_"+variable_from
            boom_angle = sonics[
                    (sonics['Mast']==tower["id"]) & 
                    (sonics['height']==height)]['Mast'].values[0]
            variables_config.append( {
                    'name_to': variable_to, 
                    'name_from': variable_from, 
                    'apply':f_apply,
                    'boom_angle':boom_angle})
            
        #config for a turbine
        tower_conf = {'type': 'mast', 
                        'id': str(tower['id']), 
                        'long_name': 'Twr' + str(int(tower['id'])), 
                        'latitude': tower['lat'], 
                        'longitude': tower['lon'], 
                        'altitude': height,
                        'variables': variables_config
                    }
        config['parameters']['positions'].append(tower_conf)
    

Finally add a custom function to process time. 

In [6]:
config['parameters']['time_apply'] = fix_time

## 3. Convert he data

In [7]:
builder = Builder(config_file = r'Alaiz.yaml',config=config)
builder.build()

|  Processing ALEX17_201808_20Hz_sample.csv ...
|  Writing to output/ALEX17_201808_20Hz_sample.nc .
If using for the first time, then you should add the metadata of the variable to an appropriate .json file in the lidaco/variables folder.
It is YOUR responsibility to do that.
If using for the first time, then you should add the metadata of the variable to an appropriate .json file in the lidaco/variables folder.
It is YOUR responsibility to do that.


## 4. Using the generated dataset
### 4.1 Load the file

In [8]:
from netCDF4 import Dataset
output_file = Dataset("./output/ALEX17_201808_20Hz_sample.nc", "r", format="NETCDF4")

In [9]:
output_file

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    conventions: CENER
    version: 1.0
    title: 
    creator: 
    references: 
    site: Alaiz
    general_comment: 
    institution: DTU,CENER
    data_processing_history: 
    start_time: 2018-09-30 01:04:29
    end_time: 2018-10-04 00:48:29
    dimensions(sizes): time(576), position(19)
    variables(dimensions): float64 time(time), <class 'str'> id(position), <class 'str'> type(position), <class 'str'> long_name(position), float64 latitude(position), float64 longitude(position), float64 altitude(position), float64 air_temperature(position,time), float64 upward_air_velocity(position,time), float64 northward_wind(position,time), float64 status(position,time), float64 eastward_wind(position,time), float64 st_code(position,time)
    groups: 

### 4.2 Reading global attributes

In [10]:
for attr in output_file.ncattrs(): 
    print (attr, '=', getattr(output_file, attr)) 

conventions = CENER
version = 1.0
title = 
creator = 
references = 
site = Alaiz
general_comment = 
institution = DTU,CENER
data_processing_history = 
start_time = 2018-09-30 01:04:29
end_time = 2018-10-04 00:48:29


In [11]:
getattr(output_file, 'start_time')

'2018-09-30 01:04:29'

### 4.3 Reading variable attributes

In [12]:
output_file.variables['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: seconds since 1970-01-01 00:00:00.00 UTC
    long_name: time
    comment: iec_61400-25 defines time as a complex type consisting of two integers SecondSinceEpoch and FractionOfSecond with the "1970" reference. For netCDF a single DOUBLE value with undefined reference is used. We use a mix of the two by using a single DOUBLE value with 1970 reference. NetCDF has a build in support for the metadata on the used reference date and calendar
    calendar: gregorian
unlimited dimensions: 
current shape = (576,)
filling on, default _FillValue of 9.969209968386869e+36 used

### 4.4 Reading position metadata
$postion$ usally defines a turbine, or a specific height of a met mast. There can be many variables assigned to a specific postion.

In [13]:
position = 5 ### internal netcdf id which can be ignored after the data is imported
print("Type: " + output_file.variables['type'][position])
print("Id: " + output_file.variables['id'][position])
print("Long_name: " + output_file.variables['long_name'][position])
print("Latitude: " + str(output_file.variables['latitude'][position]))
print("Longitude: " + str(output_file.variables['longitude'][position]))
print("Altitude: " + str(output_file.variables['altitude'][position]))

Type: mast
Id: 3.0
Long_name: Twr3
Latitude: 42.74385844765364
Longitude: -1.5633646394797014
Altitude: 10.0


### 4.5 Reading sensor data

In [14]:
output_file.variables['air_temperature'][position].data

array([16.4857, 16.4381, 16.5583, 16.7578, 16.881 , 16.3258, 15.8523,
       15.5586, 15.8288, 15.9727, 16.1357, 16.2157, 16.3728, 16.2742,
       15.9104, 14.8888, 14.616 , 15.438 , 14.956 , 14.3509, 14.4404,
       14.6816, 14.7265, 13.4023, 13.5577, 14.4045, 13.676 , 13.3706,
       13.9462, 13.4331, 13.0538, 12.8131, 12.6066, 12.437 , 12.4147,
       12.601 , 12.0153, 12.4023, 12.657 , 11.7943, 11.6864, 11.8134,
       11.6899, 11.6252, 11.6179, 11.1849, 11.5723, 11.8222, 11.8538,
       12.0585, 12.3775, 12.9415, 13.1808, 13.4989, 14.2299, 14.6145,
       15.1961, 15.7776, 16.2832, 17.014 , 17.4698, 17.9592, 18.3573,
       18.6043, 19.1932, 19.3658, 19.629 , 19.7152, 19.879 , 20.17  ,
       20.7239, 21.1598, 21.4201, 21.3525, 21.614 , 21.8497, 22.1283,
       21.9926, 22.11  , 22.1579, 22.0876, 22.158 , 22.055 , 22.2324,
       22.0454, 21.9571, 22.001 , 22.1824, 22.1238, 21.8109, 21.9597,
       21.6625, 21.6963, 21.5037, 21.4051, 21.1181, 20.8422, 20.7787,
       20.5441, 20.3